In [102]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import math
from sklearn.preprocessing import MinMaxScaler
from scipy.ndimage import rotate
from shapely.geometry import Polygon 
import folium
from numpy import array
import pickle as pkl
import collections
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from scipy.spatial import distance
from numpy.linalg import inv
import re
import json

# Utility functions

In [2]:
def get_width_height_angle(covs, nstd=3):
    """
    Returns ellipse width, height, angle.
    
    Scaled by the factor nstd.
    """
    
    def eigsorted(cov):
        vals, vecs = np.linalg.eigh(cov)
        order = vals.argsort()[::-1]
        return vals[order], vecs[:,order]

    vals, vecs = eigsorted(covs)
    angle = np.degrees(np.arctan2(*vecs[:,0][::-1]))
    # Width and height are "full" widths, not radius
    width, height = 2 * nstd * np.sqrt(vals)
    
    return width, height, angle

def get_y(x, Cx, Cy, a, b):
    '''
    x  - coordinate in range [Cx-a, Cx+a]
    Cx - ellipse center on x
    Cy - ellipse center on y
    a  - Major axis radius (x)
    b  - Minor axis radius (y)
    '''
    # since sqrt we consider + and -
    y_plus = Cy + (np.sqrt(1 - ((x-Cx)**2)/(a**2)) * b)
    y_minus = Cy - (np.sqrt(1 - ((x-Cx)**2)/(a**2)) * b)
    return np.stack((y_plus, y_minus), axis = 1)

def rotate_point(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy

def rotate_array(p, origin=(0, 0), degrees=0):
    '''
    Rotate 2D array to degrees with around given origin.
    
    '''
    angle = np.deg2rad(degrees)
    R = np.array([[np.cos(angle), -np.sin(angle)],
                  [np.sin(angle),  np.cos(angle)]])
    o = np.atleast_2d(origin)
    p = np.atleast_2d(p)
    return np.squeeze((R @ (p.T-o.T) + o.T).T)

def normalize_points(points_2D, scaler):
    '''
    Normalize cluster (x, y) array into real world map scale.
    '''
    dummy_columns = np.zeros((points_2D[0].shape[0], 2), dtype=float)
    rotated_clusters_np = np.array(points_2D)
    N = len(points_2D)
    normalized_points = []
    for i in range(N):
        # dummy 2D columns horizontal stack because scaler dimension is 4D
        cluster_reformed = np.hstack((dummy_columns, rotated_clusters_np[i]))
        cluster_unscaled = scaler.inverse_transform(cluster_reformed)
        cluster_unscaled_2D = cluster_unscaled[:, 2:]
        normalized_points.append(cluster_unscaled_2D.tolist())
    return normalized_points

def get_vertices_ellipse(cluster_means_2D, cluster_covs_2D, nstd=3):
    '''
    Get vertices for all 10(or less) ellipses goven means 2D and covs 2D
    
    Maximum 10 clusters.
    '''
    cluster_verts = []
    
    colors = ['red', 'blue', 'green', 'yellow', 'magenta', 'cyan', 'peru', 'pink', 'teal', 'olive']
    
    N =len(cluster_means_2D)

    for i in range(N):
        center_x, center_y = cluster_means_2D[i]
        width, height, angle = get_width_height_angle(cluster_covs_2D[i], nstd)
        ellipse = Ellipse(xy=(center_x, center_y), width=width, height=height,
                         alpha=0.5, color=colors[i], angle=angle)

        vertices = ellipse.get_verts() 
        cluster_verts.append(np.array(vertices))
    return cluster_verts

def plot_data_points(xy, xtest_MF, soft_cluster_assignment_M, soft_cluster_assign_probs_M, 
                     soft_cluster_assignments_probas_raw,
                     target_user_indices, mapit):
    '''
    Input:
        xy - user location coordinates (lat, lon)
        mapit - folium Map object
    
    Return:
        mapit - folium Map object with user location markers. 
    '''
    for i, coord in enumerate(xy):
        if i in target_user_indices:
            html = f"""<div style="font-family: arial">
                   Target user_id:  {str(xtest_MF[i][-1])} <br>
                   Clusters: {str(soft_cluster_assignment_M[i])} <br>
                   Clusters log likelihood: {str(soft_cluster_assign_probs_M[i])} <br>
                   Clusters max_posterior_prob and next_max_proba : {str(soft_cluster_assignments_probas_raw[i].tolist())}
                   </div>
                """

            iframe = folium.IFrame(html,
                               width=200,
                               height=200)

            popup = folium.Popup(iframe,
                             max_width=250)

            folium.Marker(location=[ coord[0], coord[1] ], icon=folium.Icon(color="red", icon="user"),
                          popup=popup).add_to(mapit)
            
#             folium.Marker( location=[ coord[0], coord[1] ], 
#                           popup=xtest_MF[i][-1] + " in clusters: " + str(soft_cluster_assignment_M[i]),
#                          icon=folium.Icon(color="red", icon="user")).add_to( mapit )
        elif not ([coord[0], coord[1]] == xtest_MF[target_user_indices][:, 2:4]).all(axis = 1).any():
            folium.Marker( location=[ coord[0], coord[1] ], 
                          popup=xtest_MF[i][-1] + " in clusters: " + str(soft_cluster_assignment_M[i]),
                         icon=folium.Icon(color="green", icon="user")).add_to( mapit )
    return mapit

def plot_clusters(normalized_cluster_points, cluster_covs_2D, cluster_means_unscaled_2D, mapit):
    '''
    Plots cluster ellipses. Maximum 10 clusters (if more then define more collors)
    
    Input:
        normalized_cluster_points - ellipse countour location unscaled in real (lat, lon) array format.
        mapit - folium Map object
    Return:
        mapit - folium Map object with cluster ellipses.. 
    '''
    N = len(normalized_cluster_points)
    colors = ['red', 'blue', 'green', 'yellow', 'magenta', 'cyan', 'peru', 'pink', 'teal', 'olive']

    for i in range(N):
        folium_poly = normalized_cluster_points[i]
        stds = np.diag(cluster_covs_2D[i])
        means = cluster_means_unscaled_2D[i]
        print("Cluster Id: ", i)
        print("Means: ", means)
        print("Stds: ", stds)
        print('--------------------------')

        html = f"""<div style="font-family: arial">
                   Cluster id:  {str(i)} <br>
                   Color: {str(colors[i])} <br>
                   Means: {str(means)} <br>
                   Stds: {str(stds)}
                   </div>
                """

        iframe = folium.IFrame(html,
                           width=200,
                           height=200)

        popup = folium.Popup(iframe,
                         max_width=250)

        folium.Polygon(folium_poly, fill_color=colors[i],
                      popup=popup).add_to(mapit)
    return mapit

# Model, test data, cluster assignments and probas

In [3]:
with open('model.pkl','rb') as f:
     model = pkl.load(f) 
        
with open('model_params.pkl','rb') as f:
     model_params = pkl.load(f) 

with open('xtest_MF.pkl','rb') as f:
     xtest_MF = pkl.load(f)
        
with open('soft_cluster_assignments_M.pkl','rb') as f:
     soft_cluster_assignments_M = pkl.load(f) 
    
with open('soft_cluster_assignments_probas.pkl','rb') as f:
     soft_cluster_assignments_probas_raw = pkl.load(f, encoding='latin-1') 
        
with open('cluster_assign_probs_M.pkl','rb') as f:
     cluster_assign_probs_M = pkl.load(f, encoding='latin-1')
        
with open('soft_cluster_assign_probs_M.pkl','rb') as f:
     soft_cluster_assign_probs_M = pkl.load(f, encoding='latin-1')

In [39]:
soft_cluster_assignments_probas_raw

array([[[0.74997147, 0.74997147]],

       [[0.73432536, 0.73432536]],

       [[0.73124433, 0.73124433]],

       ...,

       [[0.75258371, 0.75258371]],

       [[0.73279406, 0.73279406]],

       [[0.7358383 , 0.7358383 ]]])

In [4]:
# Data prep

# Prep data
feat_scales = model_params['feat_scales']
feat_mins = model_params['feat_mins']

# Rescaling test data:
scaler = MinMaxScaler()
scaler.scale_ = feat_scales
scaler.min_ = feat_mins

cluster_means = np.array(model_params["means"])
cluster_covs = np.array(model_params["covariances"])
cluster_means_2D = cluster_means[:, 2:]
cluster_covs_2D = cluster_covs[:, 2:, 2:]
cluster_means_unscaled_2D = scaler.inverse_transform(cluster_means)[:, 2:]
xtest_MD = np.asarray(xtest_MF[:, :4], dtype=np.float64)
xtest_MD_scaled = scaler.transform(xtest_MD)

target_user_id = 'david.joy@birmingham.gov.uk'
target_user_indices = np.where(xtest_MF[:, -1] == target_user_id)

# Normalize clusters
cluster_verts = get_vertices_ellipse(cluster_means_2D, cluster_covs_2D, nstd=1)
normalized_verts = normalize_points(cluster_verts, scaler)

# Plot feature data and model (model_1645315200000)

In [5]:
SRC_SITE = 'devleader1'

def get_escl(url):
    """
    Creates ElasticSearch instance and connects to url.

    :param url:
    :type url:
    :return:
    :rtype:
    """
    escl = Elasticsearch(hosts=url, timeout=30)
    if escl.ping():
        return escl
    return False

src_url = "https://cyglass:cyglass@" + SRC_SITE + ".cyglass.com:9200/"
src_escl = get_escl(src_url)

In [6]:
feature_docs = {}
index = 'saas_gmm_table_v_gmm'

query = {
      "query": {
        "term": {
          "_id": {
            "value": "feature_data_map_1645315200000"
          }
        }
      }
}

start_page_idx = None
current_page_idx = None
current_page_size = None


for doc in scan(client=src_escl, index=index, query=query):
    docjo = doc['_source']
    start_page_idx = docjo['start_page_idx']
    current_page_idx = docjo['current_page_idx']
    current_page_size = docjo['current_page_size']
    

feature_data_ids = []
for i in range(start_page_idx, current_page_idx + 1):
    feature_data_ids.append("feature_data_1645315200000_" + str(i))
    

query_feature = {
  "query": {
    "terms": {
      "_id": feature_data_ids
    }
  }
}



all_zip_lat_lon = []

for doc in scan(client=src_escl, index=index, query=query_feature):
    docjo = doc['_source']
    doc_id = doc['_id']
    if doc_id == feature_data_ids[-1]:
        lat = docjo['latitude']
        lon = docjo['longitude']
        zip_lat_lon = list(zip(lat, lon))[:current_page_size]
        all_zip_lat_lon += zip_lat_lon
    else:
        lat = docjo['latitude']
        lon = docjo['longitude']
        zip_lat_lon = list(zip(lat, lon))
        all_zip_lat_lon += zip_lat_lon
    

In [36]:
feature_data_ids

['feature_data_1645315200000_0',
 'feature_data_1645315200000_1',
 'feature_data_1645315200000_2',
 'feature_data_1645315200000_3',
 'feature_data_1645315200000_4',
 'feature_data_1645315200000_5',
 'feature_data_1645315200000_6',
 'feature_data_1645315200000_7',
 'feature_data_1645315200000_8',
 'feature_data_1645315200000_9',
 'feature_data_1645315200000_10',
 'feature_data_1645315200000_11',
 'feature_data_1645315200000_12',
 'feature_data_1645315200000_13',
 'feature_data_1645315200000_14',
 'feature_data_1645315200000_15',
 'feature_data_1645315200000_16',
 'feature_data_1645315200000_17',
 'feature_data_1645315200000_18',
 'feature_data_1645315200000_19',
 'feature_data_1645315200000_20',
 'feature_data_1645315200000_21',
 'feature_data_1645315200000_22',
 'feature_data_1645315200000_23',
 'feature_data_1645315200000_24',
 'feature_data_1645315200000_25',
 'feature_data_1645315200000_26',
 'feature_data_1645315200000_27',
 'feature_data_1645315200000_28',
 'feature_data_164531520

In [7]:
counter=collections.Counter(all_zip_lat_lon)

In [37]:
counter

Counter({(51.73333, -2.36667): 226,
         (51.67109, -1.28278): 74357,
         (23.03333, 72.61667): 88,
         (51.90224, -0.20256): 512,
         (52.18446, -0.68759): 402,
         (32.16167, 74.18831): 121,
         (52.48142, -1.89983): 23424,
         (52.81303, -0.16256): 270,
         (51.41363, -0.75054): 90,
         (51.50853, -0.12574): 1281,
         (23.17528, 90.20722): 27,
         (52.40656, -1.51217): 222,
         (51.24677, 0.60682): 914,
         (52.6386, -1.13169): 18,
         (53.81819, -1.45655): 408,
         (52.58547, -2.12296): 231,
         (53.00382, -2.28741): 26,
         (53.38297, -1.4659): 558,
         (53.48095, -2.23743): 389,
         (55.84436, -3.64314): 71,
         (55.95206, -3.19648): 72,
         (32.32426, 74.34974): 7,
         (51.96375, 1.3511): 399,
         (50.16812, -5.10416): 155,
         (52.53232, -3.67926): 38,
         (33.72148, 73.04329): 251,
         (51.55797, -1.78116): 5,
         (53.41058, -2.97794): 8,
      

In [15]:
mapit_feature = folium.Map( location=[52.667989, -1.464582], zoom_start=6 , tiles='cartodbpositron')

for k, v in counter.items():
    folium.Marker( location=[ k[0], k[1] ], 
                          popup="#Data points: " + str(v),
                         icon=folium.Icon(color="blue", icon="user")).add_to( mapit_feature )

In [16]:
mapit_feature

In [17]:
mapit_feature = plot_clusters(normalized_verts, cluster_covs_2D, cluster_means_unscaled_2D, mapit_feature)

Cluster Id:  0
Means:  [51.93873873 -1.41454435]
Stds:  [0.0010465  0.00101074]
--------------------------
Cluster Id:  1
Means:  [51.93327594 -1.45313837]
Stds:  [0.00101977 0.00100353]
--------------------------
Cluster Id:  2
Means:  [51.80899049 -1.40053614]
Stds:  [0.0010971  0.00100564]
--------------------------
Cluster Id:  3
Means:  [51.87015707 -1.35741734]
Stds:  [0.00104218 0.00100748]
--------------------------
Cluster Id:  4
Means:  [51.85242735 -1.44569925]
Stds:  [0.00111681 0.0010058 ]
--------------------------
Cluster Id:  5
Means:  [51.83192178 -1.42719029]
Stds:  [0.00108773 0.001005  ]
--------------------------
Cluster Id:  6
Means:  [51.87882486 -1.3951796 ]
Stds:  [0.0010229  0.00100501]
--------------------------
Cluster Id:  7
Means:  [29.33696511 73.85184782]
Stds:  [0.00309456 0.00112413]
--------------------------
Cluster Id:  8
Means:  [51.80551189 -1.36378023]
Stds:  [0.00105386 0.00100585]
--------------------------
Cluster Id:  9
Means:  [  18.01628194

In [18]:
mapit_feature

# Test data and clusters

In [19]:
# Plotting

# Data points
xtest_geo = xtest_MD[:, 2:]
xy = list(zip(xtest_geo[:, 0], xtest_geo[:, 1]))
# just initialize map location can be any valid lat lon
mapit_test = folium.Map( location=[52.667989, -1.464582], zoom_start=6 , tiles='cartodbpositron')

# Data points
mapit_test = plot_data_points(xy, xtest_MF, soft_cluster_assignments_M,
                         soft_cluster_assign_probs_M, 
                         soft_cluster_assignments_probas_raw,
                         target_user_indices[0],
                         mapit_test)

# Clusters
mapit_test = plot_clusters(normalized_verts, cluster_covs_2D, cluster_means_unscaled_2D, mapit_test)

Cluster Id:  0
Means:  [51.93873873 -1.41454435]
Stds:  [0.0010465  0.00101074]
--------------------------
Cluster Id:  1
Means:  [51.93327594 -1.45313837]
Stds:  [0.00101977 0.00100353]
--------------------------
Cluster Id:  2
Means:  [51.80899049 -1.40053614]
Stds:  [0.0010971  0.00100564]
--------------------------
Cluster Id:  3
Means:  [51.87015707 -1.35741734]
Stds:  [0.00104218 0.00100748]
--------------------------
Cluster Id:  4
Means:  [51.85242735 -1.44569925]
Stds:  [0.00111681 0.0010058 ]
--------------------------
Cluster Id:  5
Means:  [51.83192178 -1.42719029]
Stds:  [0.00108773 0.001005  ]
--------------------------
Cluster Id:  6
Means:  [51.87882486 -1.3951796 ]
Stds:  [0.0010229  0.00100501]
--------------------------
Cluster Id:  7
Means:  [29.33696511 73.85184782]
Stds:  [0.00309456 0.00112413]
--------------------------
Cluster Id:  8
Means:  [51.80551189 -1.36378023]
Stds:  [0.00105386 0.00100585]
--------------------------
Cluster Id:  9
Means:  [  18.01628194

In [20]:
mapit_test

# Mahalanobis

In [22]:
target_user_id = 'david.joy@birmingham.gov.uk'
target_user_indices = np.where(xtest_MF[:, -1] == target_user_id)

close_to_target_id_1 = 'james.couper@birmingham.gov.uk'
close_to_target_id_1_indices = np.where(xtest_MF[:, -1] == close_to_target_id_1)


far_from_target_id_1 = 'awais.riaz@birmingham.gov.uk'
far_from_target_id_1_indices = np.where(xtest_MF[:, -1] == far_from_target_id_1)

In [23]:
target_user_indices

(array([ 152,  463,  899, 1067, 1105, 1199, 1786, 2510, 2604, 2697]),)

In [24]:
close_to_target_id_1_indices

(array([  29,  320,  355,  562, 1100, 1841, 2108, 2190]),)

In [25]:
far_from_target_id_1_indices

(array([ 895, 2094, 2355, 2496, 2707, 2743]),)

In [26]:
target_lat_lon = xtest_MD_scaled[152, 2:]
close_lat_lon = xtest_MD_scaled[29, 2:]
far_lat_lon = xtest_MD_scaled[895, 2:]

In [27]:
cluster_covs_2D

array([[[ 1.04649535e-03, -3.55563013e-06],
        [-3.55563013e-06,  1.01073784e-03]],

       [[ 1.01976846e-03, -5.38447897e-06],
        [-5.38447897e-06,  1.00353127e-03]],

       [[ 1.09709640e-03, -5.21726387e-08],
        [-5.21726387e-08,  1.00563908e-03]],

       [[ 1.04218430e-03, -4.07673244e-06],
        [-4.07673244e-06,  1.00747757e-03]],

       [[ 1.11680589e-03,  2.72663601e-06],
        [ 2.72663601e-06,  1.00579965e-03]],

       [[ 1.08772578e-03,  1.53769746e-06],
        [ 1.53769746e-06,  1.00500100e-03]],

       [[ 1.02290396e-03, -4.65609170e-06],
        [-4.65609170e-06,  1.00500576e-03]],

       [[ 3.09455825e-03, -3.83971350e-05],
        [-3.83971350e-05,  1.12413184e-03]],

       [[ 1.05385852e-03, -1.27036551e-06],
        [-1.27036551e-06,  1.00585226e-03]],

       [[ 8.80692940e-02, -2.45343372e-02],
        [-2.45343372e-02,  9.15098216e-03]]])

In [28]:
cluster_means_2D

array([[0.88416314, 0.53155475],
       [0.8841107 , 0.5313794 ],
       [0.88291754, 0.5316184 ],
       [0.88350475, 0.53181431],
       [0.88333454, 0.5314132 ],
       [0.88313768, 0.5314973 ],
       [0.88358796, 0.53164274],
       [0.66718327, 0.87353163],
       [0.88288415, 0.5317854 ],
       [0.55850328, 0.05361546]])

In [29]:
cluster_covs_2D_inv = inv(cluster_covs_2D)

In [30]:
cluster_covs_2D_inv

array([[[ 9.55581843e+02,  3.36159928e+00],
        [ 3.36159928e+00,  9.89388062e+02]],

       [[ 9.80642542e+02,  5.26166875e+00],
        [ 5.26166875e+00,  9.96509384e+02]],

       [[ 9.11496935e+02,  4.72885363e-02],
        [ 4.72885363e-02,  9.94392542e+02]],

       [[ 9.59538370e+02,  3.88274769e+00],
        [ 3.88274769e+00,  9.92593643e+02]],

       [[ 8.95416679e+02, -2.42739732e+00],
        [-2.42739732e+00,  9.94240376e+02]],

       [[ 9.19351346e+02, -1.40664957e+00],
        [-1.40664957e+00,  9.95026037e+02]],

       [[ 9.77629499e+02,  4.52926023e+00],
        [ 4.52926023e+00,  9.95040161e+02]],

       [[ 3.23284915e+02,  1.10424899e+01],
        [ 1.10424899e+01,  8.89952550e+02]],

       [[ 9.48895424e+02,  1.19843049e+00],
        [ 1.19843049e+00,  9.94183305e+02]],

       [[ 4.48606509e+01,  1.20274121e+02],
        [ 1.20274121e+02,  4.31740089e+02]]])

In [133]:
GMM_model, model_params_np = instantiate_model_from_ES(model_params)
cluster_posterior_probas_MD = GMM_model.predict_proba(xtest_MD_scaled)

# TARGET -> CLUSTER_3 ( target, close belonging)
target_to_cluster_3 = distance.mahalanobis(target_lat_lon, cluster_means_2D[3], cluster_covs_2D_inv[3])
close_to_cluster_3 = distance.mahalanobis(close_lat_lon, cluster_means_2D[3], cluster_covs_2D_inv[3])
far_to_cluster_3 = distance.mahalanobis(far_lat_lon, cluster_means_2D[3], cluster_covs_2D_inv[3])

print("Maholonobis dist. Target to cluster 3 (beloning): ", target_to_cluster_3, 
      '| prob: ', cluster_posterior_probas_MD[152, 3])
print("Maholonobis dist. Close to cluster 3 (beloning): ", close_to_cluster_3,
      '| prob: ', cluster_posterior_probas_MD[29, 3])
print("Maholonobis dist. Far to cluster 3 (not beloning): ", far_to_cluster_3,
      '| prob: ', cluster_posterior_probas_MD[895, 3])
print("---------------------------------------------------")

# TARGET -> CLUSTER_9 (far from target,cloe,far)
target_to_cluster_9 = distance.mahalanobis(target_lat_lon, cluster_means_2D[9], cluster_covs_2D_inv[9])
close_to_cluster_9 = distance.mahalanobis(close_lat_lon, cluster_means_2D[9], cluster_covs_2D_inv[9])
far_to_cluster_9 = distance.mahalanobis(far_lat_lon, cluster_means_2D[9], cluster_covs_2D_inv[9])

print("Maholonobis dist. Target to cluster 9 (not beloning): ", target_to_cluster_9,
     '| prob: ', cluster_posterior_probas_MD[152, 9])
print("Maholonobis dist. Close to cluster 9 (not beloning): ", close_to_cluster_9,
     '| prob: ', cluster_posterior_probas_MD[29, 9])
print("Maholonobis dist. Far to cluster 9 (not beloning): ", far_to_cluster_9,
     '| prob: ', cluster_posterior_probas_MD[895, 9])
print("---------------------------------------------------")

# TARGET -> CLUSTER_7 (far from target,close, and close to far)
target_to_cluster_7 = distance.mahalanobis(target_lat_lon, cluster_means_2D[7], cluster_covs_2D_inv[7])
close_to_cluster_7 = distance.mahalanobis(close_lat_lon, cluster_means_2D[7], cluster_covs_2D_inv[7])
far_to_cluster_7 = distance.mahalanobis(far_lat_lon, cluster_means_2D[7], cluster_covs_2D_inv[7])

print("Maholonobis dist. Target to cluster 7 (not beloning): ", target_to_cluster_7,
     '| prob: ', cluster_posterior_probas_MD[152, 7])
print("Maholonobis dist. Close to cluster 7 (not beloning): ", close_to_cluster_7,
     '| prob: ', cluster_posterior_probas_MD[29, 7])
print("Maholonobis dist. Far to cluster 7 (beloning): ", far_to_cluster_7,
     '| prob: ', cluster_posterior_probas_MD[895, 7])
print("---------------------------------------------------")


# TARGET -> CLUSTER_1 (close from target,close, and far to far)
target_to_cluster_1 = distance.mahalanobis(target_lat_lon, cluster_means_2D[1], cluster_covs_2D_inv[1])
close_to_cluster_1 = distance.mahalanobis(close_lat_lon, cluster_means_2D[1], cluster_covs_2D_inv[1])
far_to_cluster_1 = distance.mahalanobis(far_lat_lon, cluster_means_2D[1], cluster_covs_2D_inv[1])

print("Maholonobis dist. Target to cluster 1 (highly overlapping): ", target_to_cluster_1,
     '| prob: ', cluster_posterior_probas_MD[152, 1])
print("Maholonobis dist. Close to cluster 1 (highly overlapping): ", close_to_cluster_1,
     '| prob: ', cluster_posterior_probas_MD[29, 1])
print("Maholonobis dist. Far to cluster 1 (not beloning): ", far_to_cluster_1,
     '| prob: ', cluster_posterior_probas_MD[895, 1])

Maholonobis dist. Target to cluster 3 (beloning):  0.06011263090320312 | prob:  0.7312443342126268
Maholonobis dist. Close to cluster 3 (beloning):  0.7383035065896251 | prob:  0.7342052698394573
Maholonobis dist. Far to cluster 3 (not beloning):  12.310695057990428 | prob:  1.5619285256815812e-31
---------------------------------------------------
Maholonobis dist. Target to cluster 9 (not beloning):  11.863480467910085 | prob:  1.1616721934456401e-35
Maholonobis dist. Close to cluster 9 (not beloning):  11.418915413502525 | prob:  2.7473739508441416e-33
Maholonobis dist. Far to cluster 9 (not beloning):  17.85304731052794 | prob:  4.026437438725025e-72
---------------------------------------------------
Maholonobis dist. Target to cluster 7 (not beloning):  10.81479235866809 | prob:  1.9131897118741466e-28
Maholonobis dist. Close to cluster 7 (not beloning):  11.214541625760384 | prob:  3.135641413537712e-30
Maholonobis dist. Far to cluster 7 (beloning):  0.4590831987652232 | prob:  

# Feature Data from SaaSAnomalyDetection

In [53]:
def get_feature_ids_in_range(start_page_id, end_page_id, feature_data_map):
    """
    start_page_id and end_page_id : FEAT_DATA_PREFIX + str(interval_starttime) + "_" + str(page_idx)
    @param start_page_id:
    @param end_page_id:
    @param feature_data_map:
    @return:
    """

    start_id_root = start_page_id[len(FEAT_DATA_PREFIX):]
    end_id_root = end_page_id[len(FEAT_DATA_PREFIX):]

    split_start_id = re.split(ID_DELIM, start_id_root)
    start_interval_id = FEAT_DATA_MAP_PREFIX + split_start_id[0]
    start_page_idx = split_start_id[1]  # should be 0

    split_end_id = re.split(ID_DELIM, end_id_root)
    end_interval_id = FEAT_DATA_MAP_PREFIX + split_end_id[0]
    end_page_idx = split_end_id[1]

    valid_intervals = [key for key in feature_data_map.keys() if start_interval_id <= key <= end_interval_id]

    feature_data_ids = []
    for interval in valid_intervals:  # in form of FEAT_DATA_MAP_PREFIX + str(interval_starttime)
        interval_root = interval[len(FEAT_DATA_MAP_PREFIX):]
        interval_start_idx = feature_data_map[interval]['start_page_idx']
        # If the interval is the first one, starts on the start_page_idx defined by the model's start_page_id
        if interval_root == start_interval_id:
            interval_start_idx = start_page_idx
        interval_end_idx = feature_data_map[interval]['current_page_idx']
        # if the interval is the last one, ends on the end_page_idx defined by the model's curr_page_id
        if interval_root == end_interval_id:
            interval_end_idx = end_page_idx
        # range from [interval_start_idx, interval_end_idx] (inclusive)
        feature_data_ids += [FEAT_DATA_PREFIX + interval_root + ID_DELIM + str(page_idx) for page_idx in
                             range(interval_start_idx, interval_end_idx + 1)]

    return feature_data_ids

def load_feature_data(_es, _feat_data_for_fit, features_to_load, doc_ids):
    '''
    Load all feature data between start_page_id and end_page_id:
    '''
    feature_data_range_query = {
        "query": {
            "bool": {
                "must": [
                    {
                        "term": {
                            "doc_type": "feature_data"
                        }
                    },
                    {
                        "ids": {
                            "values": doc_ids
                        }
                    }
                ]
            }
        }
    }

    for item in scan(_es, index=index, query=feature_data_range_query,
                     ):
        feat_data_page = item['_source']
        for feature in features_to_load:
            _feat_data_for_fit[feature] += feat_data_page[feature]

In [54]:
ID_DELIM = "_"
MODEL_FEATURES = ['day', 'time_of_day', 'latitude', 'longitude']
FEAT_DATA_PREFIX = "feature_data_"
FEAT_DATA_MAP_PREFIX = "feature_data_map_"
start_page_id = "feature_data_1645315200000_0"
end_page_id = "feature_data_1646524800000_0"
feature_data_map = {}
feature_data_map['feature_data_map_1645315200000'] = {
          "current_page_size" : 173,
          "current_page_idx" : 108,
          "start_page_idx" : 0,
          "doc_type" : "feature_data_map_by_interval"
}

In [64]:
pulled_feature_data = {feature: [] for feature in MODEL_FEATURES}
feature_doc_ids = get_feature_ids_in_range(start_page_id, end_page_id, feature_data_map)
load_feature_data(src_escl, pulled_feature_data, MODEL_FEATURES, feature_doc_ids)
counter=collections.Counter(zip(pulled_feature_data['latitude'], pulled_feature_data['longitude']))

In [87]:
mapit_feature = folium.Map( location=[52.667989, -1.464582], zoom_start=6 , tiles='cartodbpositron')

for k, v in counter.items():
    folium.Marker( location=[ k[0], k[1] ], 
                          popup="#Data points: " + str(v),
                         icon=folium.Icon(color="blue", icon="user")).add_to( mapit_feature )

In [88]:
mapit_feature

In [89]:
mapit_feature_with_clusters = plot_clusters(normalized_verts, cluster_covs_2D, cluster_means_unscaled_2D, mapit_feature)

Cluster Id:  0
Means:  [51.93873873 -1.41454435]
Stds:  [0.0010465  0.00101074]
--------------------------
Cluster Id:  1
Means:  [51.93327594 -1.45313837]
Stds:  [0.00101977 0.00100353]
--------------------------
Cluster Id:  2
Means:  [51.80899049 -1.40053614]
Stds:  [0.0010971  0.00100564]
--------------------------
Cluster Id:  3
Means:  [51.87015707 -1.35741734]
Stds:  [0.00104218 0.00100748]
--------------------------
Cluster Id:  4
Means:  [51.85242735 -1.44569925]
Stds:  [0.00111681 0.0010058 ]
--------------------------
Cluster Id:  5
Means:  [51.83192178 -1.42719029]
Stds:  [0.00108773 0.001005  ]
--------------------------
Cluster Id:  6
Means:  [51.87882486 -1.3951796 ]
Stds:  [0.0010229  0.00100501]
--------------------------
Cluster Id:  7
Means:  [29.33696511 73.85184782]
Stds:  [0.00309456 0.00112413]
--------------------------
Cluster Id:  8
Means:  [51.80551189 -1.36378023]
Stds:  [0.00105386 0.00100585]
--------------------------
Cluster Id:  9
Means:  [  18.01628194

In [90]:
mapit_feature_with_clusters

# ML Event Check

In [73]:
ml_query = {
 "query": {
   "term": {
     "anomtype": {
       "value": "Unusual Access Location For a User"
     }
   }
 } 
}

ml_docs_unusual_location = {}

for item in scan(src_escl, index="ml_event_v_gmm", query=ml_query):
    _id = item['_id']
    _source = item['_source']
    ml_docs_unusual_location[_id] = _source

In [103]:
print(json.dumps(ml_docs_unusual_location, indent=2))

{
  "48653eff2f667ef6e8d6b0753722d9c0_location_1646652600000": {
    "endpoints": [
      {
        "type": "USER",
        "name": "user_id",
        "value": "albert.lihundi@birminghamcitycouncil.onmicrosoft.com",
        "address": {
          "ipstr": "80.249.61.2",
          "address_space": "internal",
          "locality": "Abingdon",
          "ip": "80.249.61.2",
          "country": "United Kingdom of Great Britain and Northern Ireland",
          "primary": true
        }
      }
    ],
    "triggering_features_by_model": {
      "zscore": -17.77196437459705,
      "other_triggering_features": [
        {
          "standard_deviation": 364.7267920739075,
          "type": "time_of_day",
          "feature": "Time of Day",
          "baseline_time_of_day": 727.0863742885881,
          "value_time_of_day": 698
        }
      ],
      "model_name": "gmm_model_1645315200000",
      "main_triggering_feature": {
        "value_locality": "Dar es Salaam, Tanzania, United Republic

In [80]:
unusual_location = []
baseline_location = []
user = []
model = []
zscore = []

for event_key, event_value in ml_docs_unusual_location.items():
    user.append(event_value['endpoints'][0]['value'])
    model.append(event_value['triggering_features_by_model']['model_name'])
    zscore.append(event_value['triggering_features_by_model']['zscore'])
    baseline_location.append((event_value['triggering_features_by_model']['main_triggering_feature']['baseline_location']['lat'],
                              event_value['triggering_features_by_model']['main_triggering_feature']['baseline_location']['lon']))
    unusual_location.append((event_value['triggering_features_by_model']['main_triggering_feature']['value_location']['lat'],
                             event_value['triggering_features_by_model']['main_triggering_feature']['value_location']['lon']))

In [91]:
# Plot unusual locations

for i, loc in enumerate(unusual_location):
    folium.Marker( location=[ loc[0], loc[1] ], 
                          popup= "User: " + str(user[i]) + "<br>"
                                +"Trgerring model: " + str(model[i]) + "<br>"
                                +"Z-score: " + str(zscore[i]),
                         icon=folium.Icon(color="red", icon="user")).add_to( mapit_feature_with_clusters )
    
    folium.Marker( location=[ baseline_location[i][0], baseline_location[i][1] ], 
                          popup="Baseline loc for : " + str(user[i]) + "<br>"
                                +"(lat, lon): " + str(baseline_location[i]),
                         icon=folium.Icon(color="green", icon="info-sign")).add_to( mapit_feature_with_clusters )

In [92]:
mapit_feature_with_clusters

# Add probas and mahalanobis

In [93]:
from sklearn.mixture import GaussianMixture as mixture
import copy

In [94]:
def instantiate_model_from_ES(model_params):
    """
    Instantiate existing model given GMM parameters
    @param model_params:
    @type model_params:
    @return:
    @rtype:
    """
    model_params_np = copy.deepcopy(model_params)
    for param in model_params_np:
        if isinstance(model_params_np[param], list):
            model_params_np[param] = np.asarray(model_params_np[param])
    GMM_model = mixture(n_components=model_params_np['n_components'], covariance_type='full')
    GMM_model._set_parameters([model_params_np['weights'], model_params_np['means'],
                               model_params_np['covariances'], model_params_np['precisions_cholesky']])

    return GMM_model, model_params_np

In [95]:
GMM_model, model_params_np = instantiate_model_from_ES(model_params)

In [105]:
x_anom = np.array([[1, 698, -6.82349, 39.26951]])
x_anom_scaled = scaler.transform(x_anom)

In [136]:
cluster_posterior_probas_MD_anom = GMM_model.predict_proba(x_anom_scaled)

In [138]:
np.argmax(cluster_posterior_probas_MD_anom[0]), max(cluster_posterior_probas_MD_anom[0])

(7, 1.0)

In [139]:
x_anom_scaled[:, 2:]

array([[0.32003834, 0.71640493]])

In [141]:
x_anom_to_cluster_7 = distance.mahalanobis(x_anom_scaled[:, 2:], cluster_means_2D[7], cluster_covs_2D_inv[7])
x_anom_to_cluster_8 = distance.mahalanobis(x_anom_scaled[:, 2:], cluster_means_2D[8], cluster_covs_2D_inv[8])
x_anom_to_cluster_9 = distance.mahalanobis(x_anom_scaled[:, 2:], cluster_means_2D[9], cluster_covs_2D_inv[9])

print("x_anom to cluster 7 (baseline cluster): ", x_anom_to_cluster_7,
     '| prob: ', cluster_posterior_probas_MD_anom[0][7])
print("x_anom to cluster 8 (UK cluster, distant): ", x_anom_to_cluster_8,
     '| prob: ', cluster_posterior_probas_MD_anom[0][8])
print("x_anom to cluster 9 (North and South America cluster,  distant): ", x_anom_to_cluster_9,
     '| prob: ', cluster_posterior_probas_MD_anom[0][9])

x_anom to cluster 7 (baseline cluster):  7.882603464377497 | prob:  1.0
x_anom to cluster 8 (UK cluster, distant):  18.2823070501717 | prob:  5.5643121478442384e-58
x_anom to cluster 8 (North and South America cluster,  distant):  12.417364232479919 | prob:  3.5653600064055545e-23


# Cluster density

In [149]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [150]:
len(normalized_verts)

10

In [146]:
cluster_9_verts = normalized_verts[9]

In [184]:
polygon_9 = Polygon(cluster_9_verts)
clusters_density = np.zeros(10, dtype=int)

for i, cluster_verts in enumerate(normalized_verts):
#     print(i)
    polygon = Polygon(cluster_verts)
#     print(polygon)
    for k, v in counter.items():
        point = Point(k)
        if polygon.contains(point):
            clusters_density[i] += v

In [190]:
total_data_points = sum(counter.values())

In [203]:
total_data_points/100

1081.73

In [202]:
clusters_density

array([106529, 106530, 106523, 106522, 106529, 106529, 106523,   1136,
       106522,      3])

In [201]:
(100*clusters_density/sum(clusters_density)).tolist()

[12.483681882847051,
 12.483799068607576,
 12.482978768283909,
 12.482861582523384,
 12.483681882847051,
 12.483681882847051,
 12.482978768283909,
 0.13312302395511316,
 12.482861582523384,
 0.00035155728157160166]

In [209]:
# Trim clusters, that less than 1% of data points
num_threshold = math.floor(total_data_points/100)
trimmed_indices = np.where(clusters_density > num_threshold)
clusters_density = clusters_density[trimmed_indices]

In [210]:
clusters_density

array([106529, 106530, 106523, 106522, 106529, 106529, 106523,   1136,
       106522])